### Download model



In [1]:
!wget https://storage.googleapis.com/dm-tapnet/tapnext/bootstapnext_ckpt.npz
!wget https://storage.googleapis.com/dm-tapnet/tapnext/tapnext_ckpt.npz

--2025-05-21 22:53:49--  https://storage.googleapis.com/dm-tapnet/tapnext/bootstapnext_ckpt.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.134.207, 74.125.139.207, 173.194.210.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.134.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 776980182 (741M) [application/octet-stream]
Saving to: ‘bootstapnext_ckpt.npz’

bootstapnext_ckpt.n 100%[===================>] 740.99M   168MB/s    in 4.7s    

2025-05-21 22:53:54 (158 MB/s) - ‘bootstapnext_ckpt.npz’ saved [776980182/776980182]

--2025-05-21 22:53:54--  https://storage.googleapis.com/dm-tapnet/tapnext/tapnext_ckpt.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.134.207, 74.125.139.207, 173.194.210.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.134.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 776980182 (741M) [application/octet-

### Download dataset

In [2]:
!wget --no-check-certificate https://storage.googleapis.com/dm-tapnet/tapvid_davis.zip
!unzip tapvid_davis.zip

--2025-05-21 22:54:08--  https://storage.googleapis.com/dm-tapnet/tapvid_davis.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.134.207, 74.125.139.207, 173.194.210.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.134.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1668710491 (1.6G) [application/zip]
Saving to: ‘tapvid_davis.zip’

tapvid_davis.zip    100%[===================>]   1.55G  65.2MB/s    in 15s     

2025-05-21 22:54:23 (108 MB/s) - ‘tapvid_davis.zip’ saved [1668710491/1668710491]

Archive:  tapvid_davis.zip
   creating: tapvid_davis/
  inflating: tapvid_davis/SOURCES.md  
  inflating: tapvid_davis/README.md  
  inflating: tapvid_davis/tapvid_davis.pkl  


In [3]:
import torch
import torchvision

In [4]:
torch.__version__, torchvision.__version__

('2.6.0+cu124', '0.21.0+cu124')

In [5]:
!pip install -q git+https://github.com/google-deepmind/tapnet.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.8/373.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.8 MB/s eta 0:00:00


In [6]:
!pip install -q git+https://github.com/google-deepmind/recurrentgemma.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [7]:
!pip install "numpy<2.1.0"

In [8]:
import tqdm

In [10]:
!git clone https://github.com/google-deepmind/tapnet.git tapnet-code

Cloning into 'tapnet-code'...
remote: Enumerating objects: 1335, done.
remote: Counting objects: 100% (516/516), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 1335 (delta 388), reused 371 (delta 324), pack-reused 819 (from 2)
Receiving objects: 100% (1335/1335), 5.10 MiB | 26.89 MiB/s, done.
Resolving deltas: 100% (842/842), done.


In [11]:
cd tapnet-code

/content/tapnet-code


In [ ]:
from tapnet import evaluation_datasets

davis_dataset = evaluation_datasets.create_davis_dataset(
    davis_points_path='tapvid_davis/tapvid_davis.pkl',
    query_mode='first',
    full_resolution=False,
    resolution=(256, 256),
)

cached_dataset = []
for j, batch in enumerate(davis_dataset):
  cached_dataset.append(batch)
  print(
      'video id',
      j,
  )

### TAPNext

In [ ]:
import numpy as np
from tapnet.tapnext.tapnext_torch import TAPNext
from tapnet.tapnext.tapnext_torch_utils import restore_model_from_jax_checkpoint, tracker_certainty
import torch.nn.functional as F

In [ ]:
def run_eval_per_frame(
    model,
    batch,
    get_trackwise_metrics=True,
    radius=8,
    threshold=0.5,
    use_certainty=False,
):
  with torch.no_grad():
    pred_tracks, track_logits, visible_logits, tracking_state = model(
        video=batch['video'][:, :1], query_points=batch['query_points']
    )
    pred_visible = visible_logits > 0
    pred_tracks, pred_visible = [pred_tracks.cpu()], [pred_visible.cpu()]
    pred_track_logits, pred_visible_logits = [track_logits.cpu()], [
        visible_logits.cpu()
    ]
    for frame in tqdm.tqdm(range(1, batch['video'].shape[1])):
      # ***************************************************
      # HERE WE RUN POINT TRACKING IN PURELY ONLINE FASHION
      # ***************************************************
      (
          curr_tracks,
          curr_track_logits,
          curr_visible_logits,
          tracking_state,
      ) = model(
          video=batch['video'][:, frame : frame + 1],
          state=tracking_state,
      )
      curr_visible = curr_visible_logits > 0
      # ***************************************************
      pred_tracks.append(curr_tracks.cpu())
      pred_visible.append(curr_visible.cpu())
      pred_track_logits.append(curr_track_logits.cpu())
      pred_visible_logits.append(curr_visible_logits.cpu())
    tracks = torch.cat(pred_tracks, dim=1).transpose(1, 2)
    pred_visible = torch.cat(pred_visible, dim=1).transpose(1, 2)
    track_logits = torch.cat(pred_track_logits, dim=1).transpose(1, 2)
    visible_logits = torch.cat(pred_visible_logits, dim=1).transpose(1, 2)

    pred_certainty = tracker_certainty(tracks, track_logits, radius)
    pred_visible_and_certain = (
        F.sigmoid(visible_logits) * pred_certainty
    ) > threshold

    if use_certainty:
      occluded = ~(pred_visible_and_certain.squeeze(-1))
    else:
      occluded = ~(pred_visible.squeeze(-1))

  scalars = evaluation_datasets.compute_tapvid_metrics(
      batch['query_points'].cpu().numpy(),
      batch['occluded'].cpu().numpy(),
      batch['target_points'].cpu().numpy(),
      occluded.numpy() + 0.0,
      tracks.numpy()[..., ::-1],
      query_mode='first',
      get_trackwise_metrics=get_trackwise_metrics,
  )
  return (
      tracks.numpy()[..., ::-1],
      occluded,
      {k: v.sum(0) for k, v in scalars.items()},
  )


# @title Function for raw data to the input format {form-width: "25%"}
def deterministic_eval(cached_dataset, strided=False):
  if not strided:
    for sample in tqdm.tqdm(cached_dataset, disable=True):
      batch = sample['davis'].copy()
      # batch['video'] = (batch['video'] + 1) / 2
      batch['visible'] = np.logical_not(batch['occluded'])[..., None]
      batch['padding'] = np.ones(
          batch['query_points'].shape[:2], dtype=np.bool_
      )
      batch['loss_mask'] = np.ones(
          batch['target_points'].shape[:3] + (1,), dtype=np.float32
      )
      batch['appearance'] = np.ones(
          batch['target_points'].shape[:3] + (1,), dtype=np.float32
      )

      yield batch
  else:
    for sample in tqdm.tqdm(cached_dataset):
      batch = sample['davis'].copy()
      # batch['video'] = (batch['video'] + 1) / 2
      batch['visible'] = np.logical_not(batch['occluded'])[..., None]
      batch['padding'] = np.ones(
          batch['query_points'].shape[:2], dtype=np.bool_
      )
      batch['loss_mask'] = np.ones(
          batch['target_points'].shape[:3] + (1,), dtype=np.float32
      )
      batch['appearance'] = np.ones(
          batch['target_points'].shape[:3] + (1,), dtype=np.float32
      )
      backward_batch = {k: v.copy() for k, v in batch.items()}
      for key in ['visible', 'appearance', 'loss_mask', 'target_points']:
        backward_batch[key] = np.flip(backward_batch[key], axis=2)
      backward_batch['video'] = np.flip(backward_batch['video'], axis=1)
      backward_queries = (
          backward_batch['video'].shape[1]
          - backward_batch['query_points'][..., 0]
          - 1
      )
      backward_batch['query_points'][..., 0] = backward_queries
      yield batch, backward_batch

### Create the model and load checkpoint

In [ ]:
model = TAPNext(image_size=(256, 256))
ckpt_path = 'bootstapnext_ckpt.npz'
model = restore_model_from_jax_checkpoint(model, ckpt_path)
model.cuda()

### Run inference

In [ ]:
standard_eval_scalars_list = []
preds = []
for batch in deterministic_eval(cached_dataset):
  batch = {k: torch.from_numpy(v).cuda().float() for k, v in batch.items()}
  with torch.amp.autocast('cuda', dtype=torch.float16, enabled=True):
    tracks, occluded, scores = run_eval_per_frame(
        model, batch, get_trackwise_metrics=False, use_certainty=False
    )
  standard_eval_scalars_list.append(scores)
  preds.append((tracks, occluded))


print('')
print(
    'AJ',
    np.mean([
        standard_eval_scalars_list[k]['average_jaccard']
        for k in range(len(standard_eval_scalars_list))
    ]),
)
print(
    'OA',
    np.mean([
        standard_eval_scalars_list[k]['occlusion_accuracy']
        for k in range(len(standard_eval_scalars_list))
    ]),
)
print(
    'PTS',
    np.mean([
        standard_eval_scalars_list[k]['average_pts_within_thresh']
        for k in range(len(standard_eval_scalars_list))
    ]),
)